In [1]:
import sys
import math
sys.setrecursionlimit(1500)

In [22]:
# convert input txt into an 140000 * 28 array
def inputDigit(img_dir="digitdata/trainingimages"):
    with open(img_dir) as digitTxt:
        image = [list(line)[0:28] for line in digitTxt]
        rows = len(image)
        
    for i in range(rows):
        for j in range(28):
            if image[i][j] == ' ':
                image[i][j] = 0
            else:
                image[i][j] = 1
    return image

In [23]:
trainingImg = inputDigit("digitdata/trainingimages")
#print(len(trainingImg))
#print(len(trainingImg[139999]))

In [24]:
def readLable(name="digitdata/traininglabels"):
    with open(name) as label:
        labels = []
        for line in label:
            labels.append(line[0])
    return labels

In [25]:
trainingLable = readLable()
#print(len(trainingLable)

In [26]:
def mmse(preds, labels):
    error_sum = 0
    for i in len(labels):
        sum += (label[i] - preds[i]) ** 2
    return sum

In [27]:
def unrollImg(img):
    image_num = int(len(img)/28)     #each img consists of 28 lines
    unrolledImg = [[] for x in range(image_num)]          # 500*784
    offset = 0
    for i in range(image_num):  # i: 0--500
        for j in range(28):   # j: 0--28  roll num in rolled img
            unrolledImg[i] += img[offset+j]
        offset +=28

    return unrolledImg

In [28]:
# ZERO init 10 perceptrons' weights to
def init_weights():
    weights = [[0 for x in range(784)] for y in range(10)]
    return weights

In [29]:
def dot_product(a, b):
    return sum(i[0] * i[1] for i in zip(a, b))

In [34]:
def update(w, true_y, pred_y, x, alpha=1):
    w = w + alpha * (true_y - pred_y) * x

In [35]:
def train(weights, trainingImg, trainingLable, epoch_num=1000 ):
    alpha = 1
    # iterate epoch_num times
    for e in range(epoch_num):
        # iterate through every images
        for i in range(len(trainingImg)):
            # train all 10 perceptrons with every single image
            cur_image = trainingImg[i]
            for p in range(10):
                inference = math.copysign(1, dot_product(weights[p], cur_image))
                if trainingLable[i] == p:
                    true_y = 1
                else:
                    true_y = -1
                update(weights[p], true_y, inference, cur_image, alpha)
                
    return weights

In [36]:
#list = [[] for x in range(500)]
#print(len(list))
#print(len(list[0]))

#a = unrollImg(trainingImg)
#print(len(a))
#print(len(a[0]))

#w = init_weights()
#print(len(w))
#print(len(w[0]))

#print(math.copysign(1, -5))

In [37]:
weights = init_weights()
epoch_num = 2000
unrolled_img = unrollImg(trainingImg)
trained_weights = train(weights, unrolled_img, trainingLable, epoch_num)


TypeError: can't multiply sequence by non-int of type 'float'

In [ ]:
def conf_matrix(trainingLable, trainingImg):
    